In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [16]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=250, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# Create a heatmap for cloudiness
locations = city_data_df[['Lat', 'Lng']]
cloudiness = city_data_df['Cloudiness']

# Create figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to add the data
fig


Figure(layout=FigureLayout(height='420px'))

In [29]:
# Create a heatmap for windiness
locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df['Wind Speed']

# Create figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=100, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to add the data
fig


Figure(layout=FigureLayout(height='420px'))

In [31]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [32]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Vaitape,PF,2022-06-22 23:34:37,-16.5167,-151.7500,80.35,63,9,9.13
8,8,Rikitea,PF,2022-06-22 23:34:39,-23.1203,-134.9692,75.33,77,50,16.87
13,13,Muncar,ID,2022-06-22 23:34:41,-8.4333,114.3333,78.58,86,90,8.14
14,14,Avarua,CK,2022-06-22 23:34:41,-21.2078,-159.7750,84.25,74,75,17.27
15,15,Thompson,CA,2022-06-22 23:34:42,55.7435,-97.8558,75.36,33,75,8.05
20,20,Hun,LY,2022-06-22 23:34:44,29.1268,15.9477,77.49,41,0,8.19
22,22,Robertsport,LR,2022-06-22 23:34:44,6.7533,-11.3686,79.20,80,92,8.14
23,23,Banda Aceh,ID,2022-06-22 23:34:45,5.5577,95.3222,77.92,86,100,11.92
27,27,Batavia,US,2022-06-22 23:34:46,41.8500,-88.3126,89.40,35,0,16.11
28,28,Ewa Beach,US,2022-06-22 23:34:08,21.3156,-158.0072,85.95,52,40,19.57


In [36]:
# QC data
preferred_cities_df.count()

City_ID       184
City          184
Country       184
Date          184
Lat           184
Lng           184
Max Temp      184
Humidity      184
Cloudiness    184
Wind Speed    184
dtype: int64

In [37]:
# Check for null values
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [40]:
# Create DataFrame called hotel_df to store hotel names along the city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Vaitape,PF,80.35,-16.5167,-151.7500,
8,Rikitea,PF,75.33,-23.1203,-134.9692,
13,Muncar,ID,78.58,-8.4333,114.3333,
14,Avarua,CK,84.25,-21.2078,-159.7750,
15,Thompson,CA,75.36,55.7435,-97.8558,
20,Hun,LY,77.49,29.1268,15.9477,
22,Robertsport,LR,79.20,6.7533,-11.3686,
23,Banda Aceh,ID,77.92,5.5577,95.3222,
27,Batavia,US,89.40,41.8500,-88.3126,
28,Ewa Beach,US,85.95,21.3156,-158.0072,


In [41]:
# Set parameters to search for a hotel in key-value pairs
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [44]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [45]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Vaitape,PF,80.35,-16.5167,-151.7500,Conrad Bora Bora Nui
8,Rikitea,PF,75.33,-23.1203,-134.9692,People ThankYou
13,Muncar,ID,78.58,-8.4333,114.3333,Zaky Homestay
14,Avarua,CK,84.25,-21.2078,-159.7750,Paradise Inn
15,Thompson,CA,75.36,55.7435,-97.8558,Burntwood Hotel
...,...,...,...,...,...,...
550,Kerema,PG,81.32,-7.9631,145.7785,Frena Lodge
553,Marsaxlokk,MT,76.08,35.8419,14.5431,Water's Edge
554,Najran,SA,83.05,17.4924,44.1277,محمد هادي ال فطيح
556,Seydi,TM,78.42,39.4816,62.9137,Seydi oteli


In [47]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [49]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))